# Post processing notebook 

### This notebook import the visualisation functions from the main library and loads the state and meta file and allows you to modify the appearance of the output of Affinity-VAE


In [1]:
import torch 
import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, fixed, Dropdown
from avae.vis import latent_embed_plot_umap, latent_embed_plot_tsne
from avae.utils import colour_per_class
import warnings
warnings.filterwarnings('ignore')


### The following functions take in the path of the meta and the state file respectively and load them 

In [31]:
def load_meta(meta_fn, device="cpu"):
    meta_df = pd.read_pickle(meta_fn)

    mu = meta_df[
        [col for col in meta_df.columns if col.startswith("lat")]
    ].to_numpy()  # Assuming the column name for latent variables is 'latent'
    
    labels = meta_df["id"]

    pose = meta_df[
        [col for col in meta_df.columns if col.startswith("pos")]
    ].to_numpy()
    
    std = meta_df[
        [col for col in meta_df.columns if col.startswith("std")]
    ].to_numpy()
    std = std[:,1:]

    z =  np.random.randn(*std.shape)  * std + mu
    return mu, pose, labels, std, z
    

def load_model(model_fn, device="cpu"):
    checkpoint = torch.load(model_fn, map_location=torch.device(device))
    model = checkpoint["model_class_object"]
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    model.eval()
    return model

# Enter the path for the saved model and corresponding meta file 

In [38]:
model_fn = 'path/to/model'
meta_fn = 'path/to/meta'

# Grab the Information in the meta file and load the model
### From the meta file 
1. `mu`: mean of the latents ($\mu$)
2. `std`: standard deviation of the latents ($\sigma$)
3. `p`: $pose$
4. `labels`: class labels
5. `z`: sampled latent ($z$) 
### model loaded to `model`

In [33]:
mu, p, labels, std, z = load_meta(meta_fn)
model =load_model(model_fn)

# Configure latent embedding:
#### select the following to plot the UMAP embedding of the latent space : 
1. `select_function`: The drop down list allows you to choose between UMAP and TSNE for your dimentionality reduction of the latent embeddings. 
2. `rs` : This slider sets the random state for the UMAP plot.
3. `perplexity`: This slider sets the perplexity for the TSNE plot.
4. `Data Type`: This drop down list allows you to select which variable to plot (`z` : stochastic sample of latent space, `mu`: the mean of the latent space)
5. `class_list_order`: This variable ensures that the colours used in the plot are picked in the order you ahve provided (for consistency when produing more than one plot), you can copy the list from the output of affinity-vae or leave as `None` if you dont need this variable.  
6. `marker_size`: adjust the size of markers for each data point to have the desired plot appearence.

In [36]:
# Define the slider widget
random_state_slider = IntSlider(min=0, max=100, step=1, value=42)
perplexity_slider = IntSlider(min=2, max=100, step=1, value=40, description='Perplexity')
class_list_order = ['1S3X', '3QM1', '3GL1', '3H84', '2CG9', '3D2F', '1U6G', '3CF3', '1BXN', '1QVR', '4CR2', '5MRC', 'fidu']
# Define the dropdown widget for selecting the function
function_selector = Dropdown(options=['latent_embed_plot_umap', 'latent_embed_plot_tsne'],
                             value='latent_embed_plot_umap', description='Select Function')

# Use the interact function with both widgets

interact(lambda function, data_type, rs, perplexity: (latent_embed_plot_umap(xs=z if data_type == 'z' and function == 'latent_embed_plot_umap' else (mu if data_type == 'mu' else p),
                                                                ys=labels, classes=class_list_order,  rs=rs, marker_size=100, l_w= 2.5,  display=True)
                                          if function == 'latent_embed_plot_umap'
                                          else latent_embed_plot_tsne(xs=z if data_type == 'z' and function == 'latent_embed_plot_tsne' else mu,
                                                                      ys=labels, perplexity=perplexity, marker_size=100, l_w= 2.5, display=True)),
        function=function_selector,
        data_type=Dropdown(options=['z', 'mu', 'p'], value='z', description='Data Type'),
        rs=random_state_slider,
        perplexity = perplexity_slider)

interactive(children=(Dropdown(description='Select Function', options=('latent_embed_plot_umap', 'latent_embed…

<function __main__.<lambda>(function, data_type, rs, perplexity)>

## Creating new latent interpolation plots

1. choose the size of your input images : `dsize`

2. choose the plot interpolation steps via the slider : `num_int`

note that everytime you drag the `num_int` slider the plot corners change, if you would like to start from a given number of interpolation steps change the value of `init_interpolation_steps` in the code cell below. 

In [37]:
from avae.vis import latent_4enc_interpolate_plot

dsize = [32,32]
init_interpolation_steps = 10

number_interpolation = IntSlider(min=2, max=20, step=1, value=init_interpolation_steps)
interact(lambda num_int: latent_4enc_interpolate_plot(dsize= dsize, xs= torch.from_numpy(z).to(dtype=torch.float), 
                             ys= labels, vae = model ,device = "cpu", plots_config = f"1,{num_int}", 
                             poses = p, display = True),num_int=number_interpolation)

interactive(children=(IntSlider(value=10, description='num_int', max=20, min=2), Output()), _dom_classes=('wid…

<function __main__.<lambda>(num_int)>

## Creating new pose interpolation plots
1. choose the size of your input images : `dsize`
2. choose the plot interpolation steps via the slider : `number_of_samples`
3. choose the classes you would like the interpolation to be generated for via the variable : `pose_vis_class`
4. `specific_enc` : If you do not wish to use a specific encoding, set this variable to `None`


In [ ]:
img_path = "/path/to/image"
img = NormalizeData(np.load(img_path))
plt.imshow(img)
_, specific_enc, _, _, _ = model(torch.from_numpy(img[np.newaxis,np.newaxis, ...]).to(torch.float32))     

specific_enc = specific_enc.detach().numpy()[0]

In [ ]:
from avae.vis import pose_class_disentanglement_plot

dsize= [32,32]
pose_vis_class = "1,2,3,4,5,6,7,8,9"
specific_enc = None

num_int = IntSlider(min=2, max=20, step=1, value=8)
interact(lambda number_of_samples:pose_class_disentanglement_plot(dsize= dsize, x= torch.from_numpy(z).to(dtype=torch.float), 
                                y= labels, pose_vis_class=pose_vis_class, poses = p, vae = model,
                                device = "cpu", number_of_samples = number_of_samples, specific_enc= specific_enc, display = True), number_of_samples = num_int )


# Plot Affinity Matrix 

1. `affinity_path`: The path to affinity matrix 
2. `classes`: The classes you would like to represent in the plot

In [ ]:
affinity_path = "/path/to/affinity/affinity.csv"
classes = "aebdiju2"
classes=list(classes)

lookup = pd.read_csv(affinity_path, header=0)

lookup = lookup.loc[[lookup.columns.get_loc(c) for c in classes if c in lookup],classes]


with plt.rc_context(
    {"font.weight": "bold", "font.size": int(len(classes) / 3) + 3}
):
    fig, ax = plt.subplots(
        figsize=(int(len(classes)) / 2, int(len(classes)) / 2)
    )

ax.set_title("Affinity Matrix")
im = ax.imshow(lookup, vmin=-1, vmax=1, cmap=plt.cm.get_cmap("RdBu"))
ax.set_xticks(np.arange(0, len(classes)))
ax.set_xticklabels(classes)
ax.set_yticks(np.arange(0, len(classes)))
ax.set_yticklabels(classes)

ax.tick_params(axis="x", rotation=90, labelsize=16)
ax.tick_params(axis="y", labelsize=16)

pos = ax.get_position()

# Set the height of the color bar to match the height of the plot
cbar = plt.colorbar(im)
cbar.set_ticks([-1, 0, 1])  # You can customize the ticks as needed

# Adjust the position of the color bar to match the height of the plot
cbar.ax.set_position([pos.x1 - 0.01, pos.y0, 0.02, pos.height-1])

fig.tight_layout()
plt.show()


# Plot the cosine similarity of the latent space
Choose from the dropdown menu between plotting the average or the standard deviation of the cosine similarity 

In [ ]:
from avae.vis import latent_space_similarity_plot

interact(lambda plot_mode: latent_space_similarity_plot(latent_space= z, class_labels = labels, 
                             classes_order= classes,plot_mode = plot_mode, 
                             display = True, font_size=16, dpi=100), plot_mode=Dropdown(options=["mean", "std"]))



# Plot Rotation Angle-Pose correlation    

This figure takes in an original image and rotates it through a range of angle from `THETA_LOWER` to `THETA_UPPER` and extract the pose value learnt. It then plots the corresponding pose value and the angle of rotation as well as the decoded image grid.

- `img_path`: Input the path of an image from your data 
- `THETA_LOWER`, `THETA_UPPER`: Input the lower and upper range of rotation in degrees

In [ ]:
from scipy.ndimage import rotate
from skimage.util import montage 

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

img_path = "/path/to/image"
THETA_LOWER = -30
THETA_UPPER = 30
img = NormalizeData(np.load(img_path))

r=[]
original = []
pose_angle=[]

with torch.inference_mode():
    for theta in range(THETA_LOWER,THETA_UPPER):
        image = rotate(img, theta, order=0, reshape = False)
        x, z, z_pose, mu, logvar = model(torch.from_numpy(image[np.newaxis,np.newaxis, ...]).to(torch.float32))     
        
        if theta % 5 == 0 : 
            r.append(x.squeeze().detach().numpy())
            original.append(image.squeeze())
        pose_angle.append(z_pose[0,0].cpu().clone().numpy())


In [ ]:
fig, ax = plt.subplots(1,3, figsize=(10,4))
ax1, ax2, ax3 = ax

coef = np.polyfit(range(THETA_LOWER,THETA_UPPER),np.squeeze(pose_angle),1)
poly1d_fn = np.poly1d(coef) 
ax1.plot(range(THETA_LOWER,THETA_UPPER),np.squeeze(pose_angle),'ro',range(THETA_LOWER,THETA_UPPER),poly1d_fn(range(THETA_LOWER,THETA_UPPER)),'--k',
    linewidth=2)
ax1.set_xlabel(r'$\theta$')
ax1.set_ylabel('pose')

r = np.stack(r, axis=0)
original = np.stack(original, axis=0)
m = montage(r, grid_shape=(3, 6))
n = montage(original, grid_shape=(3, 6))
ax2.imshow(m); ax3.imshow(n)
ax2.axis('off'); ax3.axis('off')
ax2.set_title(" images"); ax3.set_title("Original images")
fig.tight_layout()
